In [152]:
import sqlite3
import json
import pandas as pd
import glob
from datetime import datetime

# Make Database 

Create the database and tables for reading my logs

In [203]:
conn = sqlite3.connect("results.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS nd_model_registry (
    name TEXT NOT NULL,
    models TEXT NOT NULL,
    dataset TEXT NOT NULL,
    kfold INTEGER,
    nd_structure TEXT NOT NULL,
    model_structure TEXT NOT NULL,
    accuracy_score REAL,
    run_time_seconds REAL,
    inner_kfolds INTEGER NOT NULL,
    run_timestamp TEXT,
    notes TEXT,
    PRIMARY KEY (name, models, dataset, kfold, inner_kfolds)
    )
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS competitor_model_registry (
    name TEXT NOT NULL,
    model TEXT NOT NULL,
    dataset TEXT NOT NULL,
    kfold INTEGER,
    accuracy REAL,
    run_time_seconds REAL,
    run_timestamp TEXT,
    notes TEXT,
    PRIMARY KEY (name, model, dataset, kfold)
)
""")

conn.commit()


In [168]:
# Delete my tables
# cursor.execute("""
# DROP TABLE competitor_model_registry
# """)

# conn.commit()

In [148]:
# Delete my tables
# cursor.execute("""
# DELETE FROM nd_model_registry
# """)

# conn.commit()

In [188]:
def insert_dichotomies(cursor, all_dichotomies):
    cursor.executemany("""
        INSERT INTO nd_model_registry (
                        name,
                        models,
                        dataset,
                        kfold,
                        nd_structure,
                        model_structure,
                        accuracy_score,
                        run_time_seconds,
                        inner_kfolds,
                        run_timestamp,
                        notes)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ON CONFLICT(name, models, dataset, kfold, inner_kfolds)
    DO UPDATE SET
        nd_structure = excluded.nd_structure,
        model_structure = excluded.model_structure,
        accuracy_score = excluded.accuracy_score,
        run_time_seconds = excluded.run_time_seconds,
        run_timestamp = excluded.run_timestamp,
        notes = excluded.notes
    WHERE excluded.run_timestamp > nd_model_registry.run_timestamp;
    """, all_dichotomies)

def insert_competitors(cursor, all_competitors):
    cursor.executemany("""
        INSERT INTO competitor_model_registry (
            name,
            model,
            dataset,
            kfold,
            accuracy,
            run_time_seconds,
            run_timestamp,
            notes)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ON CONFLICT(name, model, dataset, kfold)
    DO UPDATE SET
        accuracy=excluded.accuracy,
        run_time_seconds=excluded.run_time_seconds,
        run_timestamp = excluded.run_timestamp,
        notes = excluded.notes
    WHERE excluded.run_timestamp > competitor_model_registry.run_timestamp;
    """, all_competitors)

# Fill ND table results

In [90]:
all_datasets = [
    # 'letter_recognition',
    'car_evaluation',
    'mfeat-factors',
    'mfeat-fouriers',
    'mfeat-karhunen',
    'mfeat-morphological',
    'mfeat-pixel',
    'mfeat-zernlike',
    'optdigits',
    'pageblocks',
    'handwritten_digits',
    'satimage',
    'image_segment',
    'beans_data',
]

In [227]:
models = "svm"
name_add = "inner_kfold_"
log_path = r"C:\Users\maxdi\OneDrive\Documents\uni_honours_docs\getting_paper_ready\my models\randomforest\logs\outter_kfolds\\*"
log_path = r"C:\Users\maxdi\OneDrive\Documents\uni_honours_docs\getting_paper_ready\my models\svm\logs\inner_kfolds\\*"
includes_inner_kfolds = 1
all_files = glob.glob(log_path)
notes = ""

In [228]:
to_be_added = []
new_kfold = 0
for dataset in all_datasets:
    data = []
    insert_data = []
    print(f"Starting dataset: {dataset}")
    my_file = [file for file in all_files if dataset in file]
    new_kfold = 0
    
    for log_file_name in my_file:
        with open(log_file_name, 'r') as file:
            for line in file:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                    
    data = sorted(data, key=lambda x: datetime.strptime(x["timestamp"], "%Y-%m-%d %H:%M:%S"))
    
    for line_num in range(0,len(data)):
        # Looking for line: Model diagram saved here:
        line = data[line_num]
        if ("inner_kfolds" in line['message']) == includes_inner_kfolds:
            pass
        if dataset not in line['message']:
            continue
        if models.lower() not in line['message'].lower():
            continue
        if "Model diagram saved here: models/plot_" in line['message']:
            name = name_add + dataset + "_" + models
            kfold_num = line['message'].find(dataset) - 1 - includes_inner_kfolds
            new_kfold = line['message'][kfold_num]
            # if kfold > new_kfold:
            #     insert_data = []
            kfold = line['message'][kfold_num]
            models_used = data[line_num-5]['message']
            nd_structure = data[line_num-6]['message']
            run_time_log = data[line_num-7]['message']
            run_time = ''.join(c for c in run_time_log if c.isdigit() or c == '.').strip(".")
            accuracy = data[line_num+1]['message'].split(" ")[-1]
            timestamp = line["timestamp"]
            insert_string = [name, models, dataset, kfold, nd_structure, models_used, accuracy, run_time, includes_inner_kfolds, str(timestamp), notes]
            insert_data.append(insert_string)
            if str(kfold) == "5":
                print("Found all fold scores!")
                insert_dichotomies(cursor, insert_data)
                break

    conn.commit()
    # Convert to DataFrame
    # df = pd.DataFrame(data)

Starting dataset: car_evaluation
Found all fold scores!
Starting dataset: mfeat-factors
Found all fold scores!
Starting dataset: mfeat-fouriers
Found all fold scores!
Starting dataset: mfeat-karhunen
Found all fold scores!
Starting dataset: mfeat-morphological
Found all fold scores!
Starting dataset: mfeat-pixel
Found all fold scores!
Starting dataset: mfeat-zernlike
Found all fold scores!
Starting dataset: optdigits
Found all fold scores!
Starting dataset: pageblocks
Found all fold scores!
Starting dataset: handwritten_digits
Found all fold scores!
Starting dataset: satimage
Found all fold scores!
Starting dataset: image_segment
Found all fold scores!
Starting dataset: beans_data
Found all fold scores!


# Fill Competitor

In [163]:
competitor_models = ['Multinomial', 'Random Forest', 'KNN', 'SVM OVO', 'SVM OVR',
       'LDA', 'Xgboost OVO', 'Xgboost OVR', 'Multilayer Perceptron']
file_path = r"C:\Users\maxdi\OneDrive\Documents\uni_honours_docs\getting_paper_ready\Outer_kfolds_results_for_paper.xlsx"
# Read the specific sheet into a DataFrame
df_accuracy = pd.read_excel(file_path, sheet_name='Competitors Accuracy').rename(columns={"Accuracy": "name"})
df_timing = pd.read_excel(file_path, sheet_name='Competitors Timing').rename(columns={"Timing": "name"})

df_accuracy = df_accuracy[df_accuracy['name'].str.contains("_fold")][["name"] + competitor_models]
df_timing = df_timing[df_timing['name'].str.contains("_fold")][["name"] + competitor_models]

df_accuracy

,name,Multinomial,Random Forest,KNN,SVM OVO,SVM OVR,LDA,Xgboost OVO,Xgboost OVR,Multilayer Perceptron
1,letter_fold1,0.767000,0.958500,0.954750,0.953750,0.924750,0.693000,0.947250,0.962750,0.948750
2,letter_fold2,0.779000,0.961250,0.958500,0.958000,0.930500,0.710250,0.951000,0.965250,0.952500
3,letter_fold3,0.769500,0.968000,0.959250,0.959250,0.933250,0.697000,0.950500,0.966750,0.955250
4,letter_fold4,0.773750,0.961500,0.956750,0.953000,0.926000,0.706000,0.947750,0.964250,0.951500
5,letter_fold5,0.778000,0.968000,0.956750,0.958750,0.929500,0.701000,0.948250,0.967500,0.955750
...,...,...,...,...,...,...,...,...,...,...
79,car-evaluation_fold1,0.907514,0.923981,0.901734,0.973988,0.956647,0.898844,0.991329,0.994220,0.994220
80,car-evaluation_fold2,0.927746,0.915503,0.895954,0.976879,0.947977,0.916185,0.991329,0.985549,0.991329
81,car-evaluation_fold3,0.916185,0.934607,0.878613,0.982659,0.950867,0.875723,0.997110,0.997110,0.994220
82,car-evaluation_fold4,0.907246,0.930566,0.907246,0.956522,0.942029,0.886957,0.991304,0.988406,1.000000


In [193]:
# Melt dataframes to long format
accuracy_long = df_accuracy.melt(id_vars=["name"], var_name="model", value_name="accuracy")
timing_long = df_timing.melt(id_vars=["name"], var_name="model", value_name="run_time_seconds")

# Merge the two dataframes on 'name' and 'dataset'
combined = pd.merge(accuracy_long, timing_long, on=["name", "model"])

# Extract the kfold number from the 'name' column
combined["kfold"] = combined["name"].str.extract(r"fold(\d+)").astype(int)
combined["dataset"] = combined["name"].str.replace(r"_fold\d+", "", regex=True)
combined["name"] = combined["name"].str.replace(r"_fold\d+", "", regex=True) + "_" + combined["model"]
combined["notes"] = ""
combined["run_timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Final schema
combined = combined[["name", "model", "dataset", "kfold", "accuracy", "run_time_seconds", "run_timestamp", "notes"]]

In [195]:
all_competitors_data = combined.values.tolist()
all_competitors_data
insert_competitors(cursor, all_competitors_data)
# for index, row in combined.iterrows():
#     print(list(row))
conn.commit()


In [196]:
new_df = cursor.execute("""SELECT * FROM competitor_model_registry""")

In [197]:
"""
UPDATE competitor_model_registry
SET dataset = CASE
    WHEN dataset = 'car-evaluation' THEN 'car_evaluation'
    WHEN dataset = 'Beans' THEN 'beans_data'
    WHEN dataset = 'mfeat-zernike' THEN 'mfeat-zernlike'
    WHEN dataset = 'page-blocks' THEN 'pageblocks'
    WHEN dataset = 'pendigits' THEN 'handwritten_digits'
    WHEN dataset = 'segment' THEN 'image_segment'
    WHEN dataset = 'mfeat-fourier' THEN 'mfeat-fouriers'
    -- Add more conditions as needed
    ELSE dataset -- Keeps the value unchanged for rows not matching any condition
END;

"""

In [198]:
conn.close()

# Compare results

Now I can compare all my performance to each other

In [ ]:

"""
WITH top_nds as (
	SELECT 
		name,
		models,
		dataset,
		kfold,
		nd_structure,
		model_structure,
		MAX(accuracy_score) as best_accuracy,
		run_time_seconds,
		inner_kfolds
	FROM
		nd_model_registry
    WHERE name LIKE 'new_split%'
	GROUP BY dataset, kfold
),
best_comps as (
	SELECT
		model,
		dataset,
		kfold,
		MAX(accuracy) as best_accuracy,
		run_time_seconds
	FROM 
		competitor_model_registry
	GROUP BY dataset, kfold
)
SELECT 
	best_comps.dataset,
	best_comps.kfold, 
	best_comps.model as best_competitor, 
	best_comps.best_accuracy as best_comp,
	top_nds.best_accuracy as best_nd,
	top_nds.name as best_nd,
	CASE 
		WHEN abs(top_nds.best_accuracy - best_comps.best_accuracy) < 0.001 THEN 'DRAW'
		WHEN top_nds.best_accuracy > best_comps.best_accuracy THEN 'WE WON'
		ELSE 'WE LOST'
	END AS who_won
FROM 
	best_comps
INNER JOIN top_nds ON 
	best_comps.dataset = top_nds.dataset
	AND best_comps.kfold = top_nds.kfold

"""

In [229]:
from itertools import combinations
def split_into_two_tuples(lst):
    n = len(lst)
    for r in range(1, n):  # Ensure both parts are non-empty
        for subset in combinations(lst, r):
            left = subset
            right = tuple(x for x in lst if x not in subset)
            yield left, right

In [232]:
lst = [1,2,3,4,5]
for left, right in split_into_two_tuples(lst):
    print(f"{left} vs {right}")

(1,) vs (2, 3, 4, 5)
(2,) vs (1, 3, 4, 5)
(3,) vs (1, 2, 4, 5)
(4,) vs (1, 2, 3, 5)
(5,) vs (1, 2, 3, 4)
(1, 2) vs (3, 4, 5)
(1, 3) vs (2, 4, 5)
(1, 4) vs (2, 3, 5)
(1, 5) vs (2, 3, 4)
(2, 3) vs (1, 4, 5)
(2, 4) vs (1, 3, 5)
(2, 5) vs (1, 3, 4)
(3, 4) vs (1, 2, 5)
(3, 5) vs (1, 2, 4)
(4, 5) vs (1, 2, 3)
(1, 2, 3) vs (4, 5)
(1, 2, 4) vs (3, 5)
(1, 2, 5) vs (3, 4)
(1, 3, 4) vs (2, 5)
(1, 3, 5) vs (2, 4)
(1, 4, 5) vs (2, 3)
(2, 3, 4) vs (1, 5)
(2, 3, 5) vs (1, 4)
(2, 4, 5) vs (1, 3)
(3, 4, 5) vs (1, 2)
(1, 2, 3, 4) vs (5,)
(1, 2, 3, 5) vs (4,)
(1, 2, 4, 5) vs (3,)
(1, 3, 4, 5) vs (2,)
(2, 3, 4, 5) vs (1,)
